# 課題（Omnicampus上で解答を提出してください）

下記の「#common」で始まるセルの中で指定されたリンク先にある購買データをベースに以下の処理を実行し、問いに答えて課題を提出してください。（データの前処理は、以下の記載の通りに行ってください。）
    
処理内容

1. `#common`のセルで事前に前処理されたデータ`target_online_retail_data_tb`を用います。
2. 購買者（`CustomerID`）の各合計購買金額を算出し、さらに金額をベースに降順に並び替えをします。  
3. カラムが`CustomerID`と合計金額のあるテーブルを作成してください。  
4. そこから、購買者を合計金額をベースに$n$等分（$n$はデータ数を越えず、分位数に同一の値が存在しない自然数）にグループ分けします。  その際、合計金額が大きいグループから順に「グループ1」「グループ2」$\cdots$「グループn」とします。
5. それぞれのグループの金額合計値を算出してください（このアプローチをデシル分析といいます）。
6. それぞれのグループが売上の何割を占めるかを求めてください。その際、「グループ1」〜「グループn」の順に並べ、`Pandas.Series`形式で回答してください。


(なお、マーケティング戦略では、このように顧客を分けることをセグメンテーションといい、上位2割に絞ってアプローチを仕掛けることをターゲティングといいます。それぞれの戦略によりますが、優良顧客に的を絞った方が投資対効果が高いことが多いため、このようなアプローチを取ることがあります。これに関連して、上位2割の顧客が売上全体の8割を占めることを「パレートの法則」と言います。)

提出するときは、以下の点に注意してください。  
>- 以下の関数`homework`の`!!WRITE ME!!`に処理を書いてください。(**「`!!WRITE ME!!`」は消して、記入してください。**)
>- 実際の提出は記述された`homework`関数全てになり、**提出場所はOmnicampus右上にある青字のHomeworkドロップダウンメニューからSaveしてください。Solutionに関数を貼り付けてからSaveを押してください。**
>- 前処理は以下のコードを使ってください。  
>- 返り値が`Pandas.Series`形式になるようにしてください。  
>- 解答は、`my_result=pd.Series([0.6,0.2,0.1,0.025,0.025,0.022,0.021,0.02,0.01,0]) `のような形になります。(売上割合はグループ順にして、合計は1になるようにします。)

In [0]:
# common
import pandas as pd
import numpy as np

## init part(データの読み込みと前処理)
file_url = "http://archive.ics.uci.edu/ml/machine-learning-databases/00352/Online%20Retail.xlsx"
online_retail_data = pd.ExcelFile(file_url)
online_retail_data_table = online_retail_data.parse('Online Retail')

online_retail_data_table['cancel_flg'] = online_retail_data_table.InvoiceNo.map(lambda x:str(x)[0])

# InvoiceNoの先頭が5であるものとIDがNullでないものが対象
target_online_retail_data_tb = online_retail_data_table[(online_retail_data_table.cancel_flg == '5') 
                                                        & (online_retail_data_table.CustomerID.notnull())]

target_online_retail_data_tb = target_online_retail_data_tb.assign(TotalPrice=target_online_retail_data_tb.Quantity * target_online_retail_data_tb.UnitPrice)

In [0]:
def homework(target_online_retail_data_tb, n):
  sum_price=target_online_retail_data_tb.groupby("CustomerID")["TotalPrice"].sum()
  df=pd.DataFrame(sum_price)
  df=df.sort_values("TotalPrice",ascending=False)
  label=[]
  for i in range(n):
    label.append("{}".format(n-i))
  group=pd.qcut(df["TotalPrice"], n, labels=label)
  group=group.rename()
  result=df.groupby(group).sum()
  result=result.sort_index(ascending=False)
  sum=result["TotalPrice"].sum()
  my_result=result["TotalPrice"]/sum

  return my_result

In [0]:
my_result=homework(target_online_retail_data_tb,10)
print(type(my_result))
my_result

<class 'pandas.core.series.Series'>


1     0.613751
2     0.132421
3     0.081744
4     0.055182
5     0.038579
6     0.028380
7     0.020183
8     0.014826
9     0.009779
10    0.005156
Name: TotalPrice, dtype: float64

In [0]:

my_result.sum()

1.0

In [0]:
pd.Series([0.6,0.2,0.1,0.025,0.025,0.022,0.021,0.02,0.01,0])

0    0.600
1    0.200
2    0.100
3    0.025
4    0.025
5    0.022
6    0.021
7    0.020
8    0.010
9    0.000
dtype: float64

In [0]:
target_online_retail_data_tb = pd.DataFrame({ 'CustomerID' : [100,101,102,103,104,105,106],'TotalPrice' : [5,6,7,8,9,10,11]})

In [0]:
target_online_retail_data_tb.head()

,CustomerID,TotalPrice
0,100,5
1,101,6
2,102,7
3,103,8
4,104,9


In [0]:
sum_price=target_online_retail_data_tb.groupby("CustomerID")["TotalPrice"].sum()
sum_price

CustomerID
100     5
101     6
102     7
103     8
104     9
105    10
106    11
Name: TotalPrice, dtype: int64

In [0]:
df=pd.DataFrame(sum_price)
df

,TotalPrice
CustomerID,
100,5
101,6
102,7
103,8
104,9
105,10
106,11


In [0]:
df=df.sort_values("TotalPrice",ascending=False)
df

,TotalPrice
CustomerID,
106,11
105,10
104,9
103,8
102,7
101,6
100,5


In [0]:
label=[]
for i in range(4):  #n+1
  label.append("{}".format(4-i))
group=pd.qcut(df["TotalPrice"], 4, labels=label) #n
group

CustomerID
106    1
105    1
104    2
103    3
102    3
101    4
100    4
Name: TotalPrice, dtype: category
Categories (4, object): [4 < 3 < 2 < 1]

In [0]:
group=group.rename()
group

CustomerID
106    1
105    1
104    2
103    3
102    3
101    4
100    4
dtype: category
Categories (4, object): [4 < 3 < 2 < 1]

In [0]:
pd.value_counts(group)

1    2
3    2
4    2
2    1
dtype: int64

In [0]:
result=df.groupby(group).sum()
result

,TotalPrice
4,11
3,15
2,9
1,21


In [0]:
result=result.sort_values("TotalPrice",ascending=False)
print(type(result))
result

<class 'pandas.core.frame.DataFrame'>


,TotalPrice
1,21
3,15
4,11
2,9


In [0]:
sum=result["TotalPrice"].sum()
sum

56

In [0]:
my_result=result["TotalPrice"]/sum
print(type(my_result))
my_result

<class 'pandas.core.series.Series'>


1    0.375000
2    0.160714
3    0.267857
4    0.196429
Name: TotalPrice, dtype: float64

In [0]:
my_result=my_result.rename()
my_result

1    0.375000
2    0.160714
3    0.267857
4    0.196429
dtype: float64